In [1]:
%run __init__.py
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os

In [3]:
pnp_results = np.load('../calib_output/pnp_results.npy', allow_pickle=True).item()
pnp_keys = list(pnp_results.keys())

In [4]:
azure_depth_folder_path = "/home/konstantin/datasets/bandeja-sequence/2021_02_17/2021-02-17-16-07-15_chessboard_plane/_azure_depth_image_raw"
azure_depth_names = sorted(os.listdir(azure_depth_folder_path), key=str.lower)[:-2]

In [5]:
pnp_timestamps = np.array([int(i) for i in pnp_keys])
azure_depth_timestamps = np.array([int(i.split('.')[0]) for i in azure_depth_names])

In [6]:
diff = np.abs(pnp_timestamps.reshape(-1, 1) - azure_depth_timestamps.reshape(1, -1))
nn_indices = np.argmin(diff, axis=-1)

In [7]:
sel_azure_depth_names = [azure_depth_names[i] for i in nn_indices]

In [8]:
from utils.io_utils import get_depth

depths = get_depth(azure_depth_folder_path, file_name_list=sel_azure_depth_names)

In /home/konstantin/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/konstantin/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/konstantin/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/konstantin/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/konstantin/.local/lib/python3.6/site-packages/m

In [9]:
from utils.io_utils import pointcloudify_depths

intrinsics = np.array([[503.9581604003906, 0.0, 325.4591064453125],
                       [0.0, 504.0079345703125, 340.33404541015625],
                       [0.0, 0.0, 1.0]])

dist_coeff = np.array([5.268435478210449, 3.5621509552001953, -6.14773682627856e-07, -4.438921678229235e-05, 0.1832902431488037, 5.597447872161865, 5.32110071182251, 0.9697896838188171])

pointclouds = pointcloudify_depths(depths, intrinsics, dist_coeff)

In [10]:
from utils.segment_utils import filter_pointclouds, segment_pointclouds

f_pointclouds = filter_pointclouds(pointclouds, (-1.5, -1.5, 0.5), (1.5, 0.5, 2.0))
s_pointclouds = segment_pointclouds(f_pointclouds)


In [11]:
r_pointclouds = {}

for i, j, in zip(pnp_keys, sel_azure_depth_names):
    r_pointclouds[i] = s_pointclouds[j]

In [12]:
np.save('../calib_output/r_pcd.npy', r_pointclouds)